In [0]:
x = 1
y = 4
print(x+y)

5


In [0]:
list = [1, 2, 3, 4, 5]
print(list)

[1, 2, 3, 4, 5]


In [0]:
my_list = [1, 2, 3, 4, 5]
print(sum(my_list))

15


In [0]:
#average
list = [1, 2, 3, 4, 5]
print(sum(list)/len(list))

3.0


In [0]:
#tuple
tup = (1, 2 ,3 ,4, 5)
print(tup)

(1, 2, 3, 4, 5)


In [0]:
my_set = {1,2,3,4,5,6}
print(my_set)
my_set.add(7)
print(my_set)
my_set.update([11,12,13])   #Add muptiple values
print(my_set)

{1, 2, 3, 4, 5, 6}
{1, 2, 3, 4, 5, 6, 7}
{1, 2, 3, 4, 5, 6, 7, 11, 12, 13}


In [0]:
my_dict = {"name": "Alice", "age": 15}
print(my_dict["name"])
my_dict["age"] = 26
print(my_dict.keys())
print(my_dict.values())

Alice
dict_keys(['name', 'age'])
dict_values(['Alice', 26])


**Spark Session**

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("PythonDF").getOrCreate()
# Creating DataFrame from list of tuples
data = [(1, "Alice"), (2, "Bob"), (3, "Carol")]
df = spark.createDataFrame(data, ["id", "name"])
df.show()

+---+-----+
| id| name|
+---+-----+
|  1|Alice|
|  2|  Bob|
|  3|Carol|
+---+-----+



# Basic Transformations and Actions on DataFrames
- `select()` returns specified columns.
- `filter()` applies row-level conditions.
- `count()` returns the number of rows.
- `describel()` gives stats like count, mean, stddev, min, max.
- `withColumn()` adds or modifies a column.

In [0]:
# Select specific columns
df.select("name").show()

# Filter rows
df.filter(df["id"] > 1).show()

# Count rows
print(df.count())

# Describe summary statistics (numeric columns)
df.describe().show()

# Add new column with literal value
from pyspark.sql.functions import lit
df = df.withColumn ("country", lit("India"))
df.show()

+-----+
| name|
+-----+
|Alice|
|  Bob|
|Carol|
+-----+

+---+-----+
| id| name|
+---+-----+
|  2|  Bob|
|  3|Carol|
+---+-----+

3
+-------+---+-----+
|summary| id| name|
+-------+---+-----+
|  count|  3|    3|
|   mean|2.0| null|
| stddev|1.0| null|
|    min|  1|Alice|
|    max|  3|Carol|
+-------+---+-----+

+---+-----+-------+
| id| name|country|
+---+-----+-------+
|  1|Alice|  India|
|  2|  Bob|  India|
|  3|Carol|  India|
+---+-----+-------+



In [0]:
sales_data = [
    ("2024-01-01", "North", "Product A",10,200.0),
    ("2024-01-01", "South", "Product B", 5, 300.0),
    ("2024-01-02", "North", "Product A", 20, 400.0),
    ("2024-01-02", "South", "Product B", 10, 600.0),
    ("2024-01-03", "East", "Product C", 15, 375.0),
]
columns = ["date", "region", "product", "quantity", "revenue"]
sales_df = spark.createDataFrame(sales_data, columns)
sales_df.show()

+----------+------+---------+--------+-------+
|      date|region|  product|quantity|revenue|
+----------+------+---------+--------+-------+
|2024-01-01| North|Product A|      10|  200.0|
|2024-01-01| South|Product B|       5|  300.0|
|2024-01-02| North|Product A|      20|  400.0|
|2024-01-02| South|Product B|      10|  600.0|
|2024-01-03|  East|Product C|      15|  375.0|
+----------+------+---------+--------+-------+



In [0]:
from pyspark.sql.functions import sum 

total_revenue_per_product = sales_df.groupBy("product").agg(  
    sum(("revenue")).alias("total_revenue")  
)
total_revenue_per_product.show()

+---------+-------------+
|  product|total_revenue|
+---------+-------------+
|Product A|        600.0|
|Product B|        900.0|
|Product C|        375.0|
+---------+-------------+



In [0]:
total_qty_region = sales_df.groupBy("region").agg(sum("quantity").alias("by region")).show()

+------+---------+
|region|by region|
+------+---------+
| North|       30|
| South|       15|
|  East|       15|
+------+---------+



In [0]:
from pyspark.sql.functions import avg
avg_revenue_per_product = sales_df.groupBy("product").agg(avg("revenue").alias("Avg_revenue_per_Product")).show()

+---------+-----------------------+
|  product|Avg_revenue_per_Product|
+---------+-----------------------+
|Product A|                  300.0|
|Product B|                  450.0|
|Product C|                  375.0|
+---------+-----------------------+



In [0]:
from pyspark.sql. functions import avg
# Calculate average revenue per product
sales_df.groupBy("product").agg (
    (sum ("revenue") /sum("quantity")).alias("avg revenue")
).show()

+---------+-----------+
|  product|avg revenue|
+---------+-----------+
|Product A|       20.0|
|Product B|       60.0|
|Product C|       25.0|
+---------+-----------+



In [0]:
display(sales_df.groupBy("product").agg(  
    sum(("revenue")).alias("total_revenue")  
))

product,total_revenue
Product A,600.0
Product B,900.0
Product C,375.0


Databricks visualization. Run in Databricks to view.

# Top Performing Region (Highest Revenue)

In [0]:
highest_revenue = sales_df.groupBy("region").agg(sum("revenue").alias("by region")).orderBy("by region", ascending=False).show()

+------+---------+
|region|by region|
+------+---------+
| South|    900.0|
| North|    600.0|
|  East|    375.0|
+------+---------+



# Average Revenue Per Region

In [0]:
from pyspark.sql. functions import avg
display(sales_df.groupBy("region").agg(avg("revenue").alias("avg_revenue_region")))

region,avg_revenue_region
North,300.0
South,450.0
East,375.0


Databricks visualization. Run in Databricks to view.